In [1]:
import json
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import textstat
from spellchecker import SpellChecker

In [11]:
json_file_path ="Cell_Phones_and_Accessories_5.json"
df = pd.read_json(json_file_path, lines=True)

In [13]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

df = pd.read_csv("okay.csv")

top_users = df['reviewerID'].value_counts().head(500).index
top_products = df['asin'].value_counts().head(500).index

filtered_df = df[df['reviewerID'].isin(top_users) & df['asin'].isin(top_products)]

user_item_matrix = filtered_df.pivot_table(
    index='reviewerID', columns='asin', values='avg_rh_cq', fill_value=0
)

svd = TruncatedSVD(n_components=20, random_state=42)
user_factors = svd.fit_transform(user_item_matrix)
item_factors = svd.components_

predicted_matrix = np.dot(user_factors, item_factors)
predicted_df = pd.DataFrame(predicted_matrix, index=user_item_matrix.index, columns=user_item_matrix.columns)

top_n = 5
top_n_recommendations = predicted_df.apply(lambda row: row.nlargest(top_n).index.tolist(), axis=1)


recommendation_df = pd.DataFrame({
    'reviewerID': top_n_recommendations.index,
    'Top_5_Recommended_Products': top_n_recommendations.values
})

print(recommendation_df.head())


             reviewerID                         Top_5_Recommended_Products
0  A0464351OZXPUPKGI6HO  [B00C5SMPA8, B00DUFFPAW, B00KB9AWEE, B00EOXW89...
1        A1048G5XB4DPQZ  [B00NIEPAQ4, B00GXHPN1U, B00HIMOGMG, B00FZV14G...
2        A10J1NIBCBVEW1  [B00I2LRLPG, B00DRNEV9S, B00D8T9QZU, B00FX8WIU...
3        A10RFE02942D9N  [B00BT7RAPG, B00JH88NHI, B00D856NOG, B00MXWFUQ...
4        A10UHQH1YL5Q6B  [B00MXWFUQC, B00CKUBLFQ, B00JH82WRK, B00JH88NH...


In [15]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(filtered_df, test_size=0.2, random_state=42)

train_matrix = train_df.pivot_table(
    index='reviewerID', columns='asin', values='avg_rh_cq', fill_value=0
)

svd = TruncatedSVD(n_components=20, random_state=42)
user_factors = svd.fit_transform(train_matrix)
item_factors = svd.components_

predicted_matrix = np.dot(user_factors, item_factors)
predicted_df = pd.DataFrame(predicted_matrix, index=train_matrix.index, columns=train_matrix.columns)

top_n = 5
top_n_preds = predicted_df.apply(lambda row: row.nlargest(top_n).index.tolist(), axis=1)

test_truth = test_df.groupby("reviewerID")["asin"].apply(set)

hit_count = 0
total_users = 0

for user, actual_items in test_truth.items():
    if user in top_n_preds:
        predicted_items = set(top_n_preds[user])
        if predicted_items & actual_items:
            hit_count += 1
        total_users += 1

hit_ratio = hit_count / total_users if total_users > 0 else 0

print(f"Hit Ratio @ {top_n}: {hit_ratio:.4f}")


Hit Ratio @ 5: 0.1012
